# Desafio Kinea


## Importando

In [1]:
import logging
logging.level=logging.INFO

In [10]:
from typing import Dict, List
import pandas as pd

In [8]:
from ambima_connect import AmbimaConnect
from titulos import LTN, NTNB, NTNF, LFT, Priv

## Interface com API Ambima

In [11]:
def clean_list(list_to_clean:List[Dict], keep_keys):
  res = [{key : val for key, val in sub.items() if key in keep_keys} for sub in list_to_clean]
  return res

In [12]:
public = AmbimaConnect('titulos publicos').content
clean_public = clean_list(public, ['tipo_titulo','data_referencia','data_vencimento','taxa_indicativa','pu'])

In [13]:
public

[{'tipo_titulo': 'LTN',
  'expressao': 'Taxa (% a.a.)/252',
  'data_vencimento': '2018-01-01',
  'data_referencia': '2017-10-20',
  'codigo_selic': '100000',
  'data_base': '2014-01-10',
  'taxa_compra': 7.2914,
  'taxa_venda': 7.2782,
  'taxa_indicativa': 7.2843,
  'intervalo_min_d0': 7.0437,
  'intervalo_max_d0': 7.4729,
  'intervalo_min_d1': 7.0371,
  'intervalo_max_d1': 7.4615,
  'pu': 986.696496},
 {'tipo_titulo': 'NTN-F',
  'expressao': 'Taxa (% a.a.)/252',
  'data_vencimento': '2018-01-01',
  'data_referencia': '2017-10-20',
  'codigo_selic': '950199',
  'data_base': '2012-01-06',
  'taxa_compra': 7.3087,
  'taxa_venda': 7.2941,
  'taxa_indicativa': 7.3002,
  'intervalo_min_d0': 7.0639,
  'intervalo_max_d0': 7.4931,
  'intervalo_min_d1': 7.053,
  'intervalo_max_d1': 7.4774,
  'pu': 1034.826807},
 {'tipo_titulo': 'LFT',
  'expressao': 'Rentabilidade (% a.a.)/252',
  'data_vencimento': '2018-03-01',
  'data_referencia': '2017-10-20',
  'codigo_selic': '210100',
  'data_base': '200

In [23]:
[x for x in clean_public if 'tipo_titulo' not in x]

[]

In [7]:
priv = AmbimaConnect('titulos privados').content
priv_list = [x['codigo_ativo'] for x in priv]

NameError: name 'AmbimaConnect' is not defined

## Cálculo Geral

In [ ]:
for titulo in clean_public[:10]:

  if 'tipo_titulo' in titulo:
    if titulo['tipo_titulo'] == 'LTN':
      calc = LTN(
        data_venc=titulo['data_vencimento'],
        taxa=titulo['taxa_indicativa']/100,
        data_ref=titulo['data_referencia'],
        )
    elif titulo['tipo_titulo'] == 'NTN-B':
      calc = NTNB(
        data_venc=titulo['data_vencimento'],
        taxa=titulo['taxa_indicativa']/100,
        data_ref=titulo['data_referencia'],
        )
    elif titulo['tipo_titulo'] == 'NTN-F':
      calc = NTNF(
        data_venc=titulo['data_vencimento'],
        taxa=titulo['taxa_indicativa']/100,
        data_ref=titulo['data_referencia'],
        )
    elif titulo['tipo_titulo'] == 'LFT':
      calc = LFT(
        data_venc=titulo['data_vencimento'],
        taxa=titulo['taxa_indicativa']/100,
        data_ref=titulo['data_referencia'],
        )
    
  titulo['pu_calc']=calc.pu
  titulo['pm_calc']=calc.pm
  titulo['duration_calc']=calc.duration

In [3]:
for codigo in priv_list[:10]:
  deb = Priv(codigo_ativo=codigo)


NameError: name 'priv_list' is not defined

In [4]:
import requests
from io import StringIO

url_list = 'http://www.debentures.com.br/exploreosnd/consultaadados/emissoesdedebentures/caracteristicas_e.asp?op_exc=False'
results_list = requests.get(url_list)
csvString_list=results_list.content.decode('Windows 1254')
csvStringIO_list = StringIO(csvString_list)
df_list = pd.read_csv(csvStringIO_list, sep="\t", index_col=False, skiprows=[0,1,2])

In [11]:
deb_list = df_list['Codigo do Ativo'].str.replace(' ', '').tolist()

In [26]:
t = clean_public[:10]
t_l = list()
for x in t: t_l.append({
  "Tipo": x['tipo_titulo'],
  "Data Ref": x['data_referencia'],
  "PU Real": x['pu'],
  "PU Calc": x['pu_calc'],
  "PU Error": utils.check_error(x['pu_calc'], x['pu']),
  # "Duration Real": x['duration'],
  "PM Calc": x['pm_calc'],
  "Duration Calc": x['duration_calc'],
  # "Duration Error": utils.check_error(x['duration_calc'], x['duration'])
  })
pd.DataFrame.from_dict(t_l)

,Tipo,Data Ref,PU Real,PU Calc,PU Error,PM Calc,Duration Calc
0,LTN,2017-10-20,986.696496,986.696497,8.88e-08%,73.00000,48.000000
1,NTN-F,2017-10-20,1034.826807,1034.826806,1.36e-07%,73.00000,48.000000
2,LFT,2017-10-20,9162.202647,9162.202647,0.0%,132.00000,89.000000
3,LTN,2017-10-20,970.982414,970.982415,7.77e-08%,163.00000,109.000000
4,LTN,2017-10-20,954.999495,954.999495,4.07e-08%,254.00000,172.000000
5,NTN-B,2017-10-20,3127.950824,3127.639187,0.00996%,293.94781,201.439706
6,LFT,2017-10-20,9162.202647,9162.202647,0.0%,316.00000,217.000000
7,LTN,2017-10-20,938.119221,937.865335,0.0271%,346.00000,237.000000
8,LTN,2017-10-20,920.958993,920.958994,6.28e-08%,438.00000,298.000000
9,NTN-F,2017-10-20,1060.800212,1060.800207,4.38e-07%,414.62645,281.119455


In [1]:
import pyodbc 

server = 'localhost' 
database = 'mydb'
username = 'admin'
password = 'admin'
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

InterfaceError: ('IM002', '[IM002] [Microsoft][ODBC Driver Manager] Data source name not found and no default driver specified (0) (SQLDriverConnect)')

In [3]:
from getpass import getpass
from mysql.connector import connect, Error

try:
    with connect(
        host="localhost",
        user=input("Enter username: "),
        password=getpass("Enter password: "),
    ) as connection:
        create_db_query = "CREATE DATABASE online_movie_rating"
        with connection.cursor() as cursor:
            cursor.execute(create_db_query)
except Error as e:
    print(e)